# Compare Models

Create object to automated model comparisons

In [1]:
import arithmetic_datasets as ad

from tensorflow.keras.layers import Input, Lambda, Dense, concatenate
from tensorflow.keras.models import Model, Sequential

import matplotlib.pyplot as plt
import random
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import tensorflow as tf
import random
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

from tensorflow.keras.callbacks import Callback

from tensorflow.keras.models import load_model

import os

## Callback Directory Filepath

In [2]:
#WIP
cb_filepath = "checkpoints"
if not os.path.exists(cb_filepath):
    os.makedirs(cb_filepath)  

### ModelComparator

```
mc = ModelComparator()
mc.add_model(name, model)
mc.add_training_set(name, data)

mc.train_models(epochs, batches, **kwargs)

mc.compare_min(metric)
mc.compare_max(metric)

mc.history_for(model_name, training_set_name)
```

In [3]:
class SaveModelCallback(Callback):
    def __init__(self, filepath,  n=100, use_model_name=False):
        super(SaveModelCallback, self).__init__()
        self.num_epochs = 100
        self.filepath = filepath
        if use_model_name:
            self.filepath = self.filepath + "/" + self.model.name
    
    def on_epoch_end(self, epoch, logs = None):
        super(SaveModelCallback, self).on_epoch_end(epoch, logs)
        if epoch % self.num_epochs == 0:
            self.model.save(f"{self.filepath}/{epoch}.model")
            print(f"model saved: {self.filepath}/{epoch}.model")

In [4]:
import os
from tensorflow.keras.callbacks import Callback

class ModelComparator:
    def __init__(self):
        self.models = {}
        self.training_sets = {}
        self.testing_sets = {}
        self.histories = {}
        self.smc_params = { "filepath" : "checkpoints", "n" : 100 }
    
    def add_model(self, name, model):
        self.models[name] = model
    
    def add_training_set(self, name, data):
        self.training_sets[name] = data
        
    def add_testing_set(self, name, data):
        self.testing_sets[name] = data
        
    def set_smc_params(self, filepath, n):
        self.smc_params["filepath"] = filepath
        self.smc_params["n"] = n
        
    def train_models(self, **kwargs):
        for model_name in self.models:
            for tset_name in self.training_sets:
                print(f"MODEL {model_name} TRAINING ON DATASET {tset_name}")
                
                start_dir = self.smc_params["filepath"]
                fpath = f"{start_dir}/{tset_name}/{model_name}"
                nn = self.smc_params["n"]
                
                if not os.path.exists(start_dir):
                    os.makedirs(start_dir)
                if not os.path.exists(f"{start_dir}/{tset_name}"):
                    os.makedirs(f"{start_dir}/{tset_name}")
                if not os.path.exists(fpath):
                    os.makedirs(fpath)
                
                model = None
                checkpoints = [cb_filepath + '/' + model_name + "/" + name
               for name in os.listdir(fpath)]
                if checkpoints:
                    latest_cp = max(checkpoints, key=os.path.getctime )
                    print('Restoring from', latest_cp)
                    model = load_model(latest_cp)
                
                if model == None:
                    model = self.models[model_name]()
                
                smc = SaveModelCallback(filepath=fpath, n=nn)
                if "callbacks" in kwargs:
                    kwargs["callbacks"].append(smc)
                
                history = model.fit(
                    self.training_sets[tset_name][0], self.training_sets[tset_name][1],
                    **kwargs
                )
                
                self.histories[(model_name, tset_name)] = history
    
    def compare_min(self, metric):
        result = {}
        minimum = None
        min_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amin(history[metric])
            if minimum == None or minimum >= result[hkey]:
                minimum = result[hkey]
                min_name = hkey
        return result, {"name" : min_name, "value":minimum}
            
    
    def compare_max(self, metric):
        result = {}
        maximum = None
        max_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amax(history[metric])
            if maximum == None or maximum >= result[hkey]:
                maximum = result[hkey]
                max_name = hkey
        return result, {"name" : max_name, "value": maximum}
    
    def history_for(self, model_name, training_set_name):
        return self.histories[(model_name, training_set_name)]
    
    def test_models(self, use_training_sets=False, acc_range=0.5):
        if use_training_sets:
            testing_sets = self.training_sets
        else:
            testing_sets = self.testing_sets
        
        for model_name in self.models:
            for tset_name in testing_sets:
                print(f"MODEL {model_name} TESTING ON DATASET {tset_name}")
                model = self.models[model_name]()
                
                x_test = testing_sets[tset_name][0]
                y_test = testing_sets[tset_name][1]
                
                pred = model.predict(x_test)
                
                test_total = 0
                acc_count = 0
                for a in range(len(x_test)):
                    x = x_test[a]
                    y = y_test[a]
                    yp = pred[a][0]
                    print(f"\tinput={x[0]} expected_output={[y]} prediction={yp} \n\t\tdifference={yp - y} \n\t\t%-different from actual={abs(yp - y)/y}")

                    if abs(yp - y) <= acc_range:
                        acc_count = acc_count + 1

                    test_total = test_total + 1

                    print(f"\t\t accuracy={acc_count / test_total}")



### Create Models

In [5]:
def nondense_model():
    model_name = "nondense_model"
    
#     if not os.path.exists(cb_filepath + "/" + model_name):
#         os.makedirs(cb_filepath + "/" + model_name)  
    
#     checkpoints = [cb_filepath + '/' + model_name + "/" + name
#                    for name in os.listdir(cb_filepath + "/" + model_name)]
#     if checkpoints:
#         latest_cp = max(checkpoints, key=os.path.getctime )
#         print('Restoring from', latest_cp)
#         return load_model(latest_cp)
    
    # Input layer of 3 neurons 
    inp = Input(shape=(1,3))
    
    #128 layer
    d2_out = Dense(128)(inp)

    #grab first, 2nd half of the 128 layer
    d2_out_p1 = Lambda(lambda x: x[:,:,0:64])(d2_out)
    d2_out_p2 = Lambda(lambda x: x[:,:,64:128])(d2_out)

    #64 layer(s)
    d3_out = Dense(64)(d2_out_p1)
    d4_out = Dense(64)(d2_out_p2)

    #grab output nodes from both 64 layers
    d5_out = concatenate([d3_out, d4_out])
    
    o = Dense(1)(d5_out)
    
    model = Model(inp, o)
    
    model._name = model_name
    
    model.compile(
        loss="MeanSquaredError",
        metrics=['accuracy']
    )
    
    return model

def dense_model_5L():
    model_name = "dense_model_5L"
    
#     if not os.path.exists(cb_filepath + "/" + model_name):
#         os.makedirs(cb_filepath + "/" + model_name)  
    
#     checkpoints = [cb_filepath + '/' + model_name + "/" + name
#                    for name in os.listdir(cb_filepath + "/" + model_name)]
#     if checkpoints:
#         latest_cp = max(checkpoints, key=os.path.getctime )
#         print('Restoring from', latest_cp)
#         return load_model(latest_cp)
    
    model_5layer = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1024, input_shape=(1,3)),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dense(256),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1)
    ])
    
#     model.name = "dense_model"

    model_5layer._name = model_name
    model_5layer.compile(
        loss="MeanSquaredError",
        metrics=['accuracy'] #Acc not working, in testing
    )

    return model_5layer

def dense_model2():
    model_name = "dense_model2"
    
#     if not os.path.exists(cb_filepath + "/" + model_name):
#         os.makedirs(cb_filepath + "/" + model_name)  
    
#     checkpoints = [cb_filepath + '/' + model_name + "/" + name
#                    for name in os.listdir(cb_filepath + "/" + model_name)]
#     if checkpoints:
#         latest_cp = max(checkpoints, key=os.path.getctime )
#         print('Restoring from', latest_cp)
#         return load_model(latest_cp)
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(2048, input_shape=(1,3)))
    
    model.add(tf.keras.layers.Dense(1024))
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.Dense(64))
    model.add(tf.keras.layers.Dense(1))
    
#     model.name = "dense_model v2"
    model._name = model_name
    model.compile(
        loss="MeanSquaredError",
        metrics = ["accuracy"]
    )
    
    return model

## Test Model

In [6]:
# def build_model():
#     model = Sequential()

#     model.add(Input(shape=(1,2)))
#     model.add(Dense(64))
#     model.add(Dense(1))

#     model.compile(
#         loss="MeanSquaredError",
#         metrics=['accuracy']
#     )
    
#     return model

# model = build_model()

### Datasets

In [7]:
num_examples = 20000
rstart = 1
rend = 10000

# MULT
mult_setX, mult_setY = ad.gen_data_mult(num_examples, rstart, rend)
mult_setX = mult_setX.reshape(num_examples, 1, 3)

# LOG(MULT)
mult_logX = np.log(mult_setX)
mult_logY = np.log(mult_setY)

### Build ModelComparator

In [10]:
mc = ModelComparator()

mc.add_model("nondense_model", nondense_model)
mc.add_model("dense_model", dense_model_5L)
mc.add_model("dense_model v2", dense_model2)

mc.add_training_set("log normal multiply", (mult_logX, mult_logY))
mc.add_training_set("multiplication", (mult_setX, mult_setY))

mc.set_smc_params("checkpoints", 10)

## Train Data (Garbage)

In [11]:
# X = np.ones((100,2))
# y = np.ones((100,1))

# model.fit(X, y, epochs=25, batch_size=1, callbacks=[smc])
# model.predict(X)

In [12]:
# model = load_model("checkpoints/chkpt-20.model")

### Train Models

In [13]:
num_epochs = 30
batch_size = num_examples

mc.train_models(
    batch_size=batch_size,
    epochs=num_epochs
)

# new_model = dense_model_5L()
# new_model = load_model("checkpoints/chkpt-200.model")

# new_model.predict(mult_logX)
#new_model

MODEL nondense_model TRAINING ON DATASET log normal multiply
Epoch 1/30
1/1 [==============================] - 0s 3ms/step - loss: 255.5139 - accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 3ms/step - loss: 119.5056 - accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 0s 2ms/step - loss: 61.9699 - accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - 0s 3ms/step - loss: 31.6575 - accuracy: 0.0000e+00
Epoch 5/30
1/1 [==============================] - 0s 2ms/step - loss: 15.3981 - accuracy: 0.0000e+00
Epoch 6/30
1/1 [==============================] - 0s 2ms/step - loss: 7.0392 - accuracy: 0.0000e+00
Epoch 7/30
1/1 [==============================] - 0s 2ms/step - loss: 3.0446 - accuracy: 0.0000e+00
Epoch 8/30
1/1 [==============================] - 0s 2ms/step - loss: 1.3010 - accuracy: 0.0000e+00
Epoch 9/30
1/1 [==============================] - 0s 4ms/step - loss: 0.6121 - accuracy: 0.0000e+00
Epoch 10/30
1/1 [===============

Epoch 16/30
1/1 [==============================] - 0s 3ms/step - loss: 0.0511 - accuracy: 0.0000e+00
Epoch 17/30
1/1 [==============================] - 0s 2ms/step - loss: 0.0508 - accuracy: 0.0000e+00
Epoch 18/30
1/1 [==============================] - 0s 3ms/step - loss: 0.0504 - accuracy: 0.0000e+00
Epoch 19/30
1/1 [==============================] - 0s 3ms/step - loss: 0.0501 - accuracy: 0.0000e+00
Epoch 20/30
1/1 [==============================] - 0s 2ms/step - loss: 0.0497 - accuracy: 0.0000e+00
Epoch 21/30
1/1 [==============================] - 0s 2ms/step - loss: 0.0493 - accuracy: 0.0000e+00
Epoch 22/30
1/1 [==============================] - 0s 2ms/step - loss: 0.0489 - accuracy: 0.0000e+00
Epoch 23/30
1/1 [==============================] - 0s 3ms/step - loss: 0.0485 - accuracy: 0.0000e+00
Epoch 24/30
1/1 [==============================] - 0s 3ms/step - loss: 0.0480 - accuracy: 0.0000e+00
Epoch 25/30
1/1 [==============================] - 0s 2ms/step - loss: 0.0476 - accuracy: 0

1/1 [==============================] - 0s 4ms/step - loss: 1096834655191040.0000 - accuracy: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 5ms/step - loss: 1089796344643584.0000 - accuracy: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 0s 5ms/step - loss: 1027673400803328.0000 - accuracy: 0.0000e+00
Epoch 4/30
1/1 [==============================] - 0s 8ms/step - loss: 820144003612672.0000 - accuracy: 0.0000e+00
Epoch 5/30
1/1 [==============================] - 0s 5ms/step - loss: 385391878209536.0000 - accuracy: 0.0000e+00
Epoch 6/30
1/1 [==============================] - 0s 4ms/step - loss: 203628409257984.0000 - accuracy: 0.0000e+00
Epoch 7/30
1/1 [==============================] - 0s 4ms/step - loss: 279336049442816.0000 - accuracy: 0.0000e+00
Epoch 8/30
1/1 [==============================] - 0s 5ms/step - loss: 170318001864704.0000 - accuracy: 0.0000e+00
Epoch 9/30
1/1 [==============================] - 0s 8ms/step - loss: 178533167005696.0000 - acc

### compare min loss

In [14]:
models_min_loss, min_loss = mc.compare_min("loss")

In [15]:
models_min_loss

{('nondense_model', 'log normal multiply'): 0.19023551046848297,
 ('nondense_model', 'multiplication'): 1093022871715840.0,
 ('dense_model', 'log normal multiply'): 0.044911064207553864,
 ('dense_model', 'multiplication'): 158082395013120.0,
 ('dense_model v2', 'log normal multiply'): 0.022095678374171257,
 ('dense_model v2', 'multiplication'): 170318001864704.0}

## Test


In [ ]:
mc.test_models(use_training_sets=True)

MODEL nondense_model TESTING ON DATASET log normal multiply
	input=[8.56750601 7.681099   5.48893773] expected_output=[16.248605006826185] prediction=[2.4952059] 
		difference=[-13.7534] 
		%-different from actual=[0.84643567]
		 accuracy=0.0
	input=[4.20469262 8.85680336 5.48893773] expected_output=[13.061495976119344] prediction=[2.1816683] 
		difference=[-10.8798275] 
		%-different from actual=[0.8329695]
		 accuracy=0.0
	input=[8.10711747 8.40335237 5.48893773] expected_output=[16.51046984574287] prediction=[2.5903525] 
		difference=[-13.920116] 
		%-different from actual=[0.8431085]
		 accuracy=0.0
	input=[7.68937111 9.0594012  5.48893773] expected_output=[16.748772303940587] prediction=[2.6768343] 
		difference=[-14.071938] 
		%-different from actual=[0.8401773]
		 accuracy=0.0
	input=[9.16115013 7.11314211 5.48893773] expected_output=[16.2742922364925] prediction=[2.4504113] 
		difference=[-13.823881] 
		%-different from actual=[0.84943056]
		 accuracy=0.0
	input=[8.97360487 8.6

		%-different from actual=[0.844671]
		 accuracy=0.0
	input=[8.91476053 7.77148876 5.48893773] expected_output=[16.686249287514876] prediction=[2.5595996] 
		difference=[-14.126649] 
		%-different from actual=[0.8466042]
		 accuracy=0.0
	input=[8.86432272 9.19857139 5.48893773] expected_output=[18.062894110887623] prediction=[2.8589816] 
		difference=[-15.203913] 
		%-different from actual=[0.8417207]
		 accuracy=0.0
	input=[8.66302364 8.54402985 5.48893773] expected_output=[17.207053484997573] prediction=[2.6925743] 
		difference=[-14.51448] 
		%-different from actual=[0.84351915]
		 accuracy=0.0
	input=[8.33710913 8.69483716 5.48893773] expected_output=[17.031946288221555] prediction=[2.682653] 
		difference=[-14.349293] 
		%-different from actual=[0.8424928]
		 accuracy=0.0
	input=[8.94376726 9.043459   5.48893773] expected_output=[17.987226262404743] prediction=[2.83603] 
		difference=[-15.1511965] 
		%-different from actual=[0.8423309]
		 accuracy=0.0
	input=[8.56464913 6.94985646

		%-different from actual=[0.84849316]
		 accuracy=0.0
	input=[8.00869818 8.42748728 5.48893773] expected_output=[16.436185461320274] prediction=[2.5827675] 
		difference=[-13.853418] 
		%-different from actual=[0.8428609]
		 accuracy=0.0
	input=[8.58148168 8.4250779  5.48893773] expected_output=[17.006559583807032] prediction=[2.6565037] 
		difference=[-14.350056] 
		%-different from actual=[0.84379536]
		 accuracy=0.0
	input=[5.69035945 7.05703698 5.48893773] expected_output=[12.747396436021951] prediction=[1.9884493] 
		difference=[-10.758947] 
		%-different from actual=[0.84401137]
		 accuracy=0.0
	input=[9.11184513 7.675546   5.48893773] expected_output=[16.78739112776493] prediction=[2.5645814] 
		difference=[-14.222811] 
		%-different from actual=[0.84723175]
		 accuracy=0.0
	input=[6.16541785 7.68017564 5.48893773] expected_output=[13.84559349466801] prediction=[2.1836143] 
		difference=[-11.66198] 
		%-different from actual=[0.8422882]
		 accuracy=0.0
	input=[8.68033193 9.0030

	input=[9.11217612 8.33639048 5.48893773] expected_output=[17.448566603780503] prediction=[2.7062883] 
		difference=[-14.742278] 
		%-different from actual=[0.8448991]
		 accuracy=0.0
	input=[5.27811466 8.32360844 5.48893773] expected_output=[13.60172310157409] prediction=[2.206521] 
		difference=[-11.395202] 
		%-different from actual=[0.83777636]
		 accuracy=0.0
	input=[8.74353163 8.52773741 5.48893773] expected_output=[17.271269038918874] prediction=[2.6995177] 
		difference=[-14.571751] 
		%-different from actual=[0.8436989]
		 accuracy=0.0
	input=[8.87010105 6.03308622 5.48893773] expected_output=[14.903187270584528] prediction=[2.1811516] 
		difference=[-12.722035] 
		%-different from actual=[0.8536453]
		 accuracy=0.0
	input=[8.2233589  8.22309055 5.48893773] expected_output=[16.44644945064079] prediction=[2.5667782] 
		difference=[-13.879671] 
		%-different from actual=[0.84393114]
		 accuracy=0.0
	input=[8.32312289 7.54009032 5.48893773] expected_output=[15.86321320773306] pre

		%-different from actual=[0.8431464]
		 accuracy=0.0
	input=[9.03276752 4.27666612 5.48893773] expected_output=[13.309433641220487] prediction=[1.8257173] 
		difference=[-11.483717] 
		%-different from actual=[0.86282533]
		 accuracy=0.0
	input=[7.04577658 8.72955887 5.48893773] expected_output=[15.775335446438081] prediction=[2.5226939] 
		difference=[-13.252642] 
		%-different from actual=[0.8400862]
		 accuracy=0.0
	input=[6.92657703 8.80792062 5.48893773] expected_output=[15.734497655276703] prediction=[2.5240402] 
		difference=[-13.210458] 
		%-different from actual=[0.8395856]
		 accuracy=0.0
	input=[9.16293425 8.58634605 5.48893773] expected_output=[17.749280299683466] prediction=[2.766451] 
		difference=[-14.98283] 
		%-different from actual=[0.8441373]
		 accuracy=0.0
	input=[6.98933527 8.9391876  5.48893773] expected_output=[15.928522867450173] prediction=[2.5603151] 
		difference=[-13.368208] 
		%-different from actual=[0.83926225]
		 accuracy=0.0
	input=[8.96877824 8.61540

...

		%-different from actual=[0.8448236]
		 accuracy=0.0
	input=[8.76186337 7.95822719 5.48893773] expected_output=[16.72009056559704] prediction=[2.5798087] 
		difference=[-14.140282] 
		%-different from actual=[0.84570605]
		 accuracy=0.0
	input=[8.20494517 8.17891933 5.48893773] expected_output=[16.383864497867602] prediction=[2.554923] 
		difference=[-13.828942] 
		%-different from actual=[0.8440586]
		 accuracy=0.0
	input=[9.06102797 7.07918439 5.48893773] expected_output=[16.14021236339884] prediction=[2.4301522] 
		difference=[-13.710061] 
		%-different from actual=[0.849435]
		 accuracy=0.0
	input=[8.2733366  8.57621653 5.48893773] expected_output=[16.849553130370186] prediction=[2.648957] 
		difference=[-14.200597] 
		%-different from actual=[0.8427877]
		 accuracy=0.0
	input=[7.75405264 8.31703348 5.48893773] expected_output=[16.07108611552816] prediction=[2.5260787] 
		difference=[-13.545008] 
		%-different from actual=[0.8428184]
		 accuracy=0.0
	input=[8.66008068 7.87929149 5

	input=[9.08148375 7.24850407 5.48893773] expected_output=[16.32998782222179] prediction=[2.4691014] 
		difference=[-13.860888] 
		%-different from actual=[0.8487996]
		 accuracy=0.0
	input=[8.89562963 8.69097842 5.48893773] expected_output=[17.58660804432439] prediction=[2.7542288] 
		difference=[-14.83238] 
		%-different from actual=[0.8433906]
		 accuracy=0.0
	input=[7.49831587 8.88002912 5.48893773] expected_output=[16.378344988235423] prediction=[2.6136155] 
		difference=[-13.76473] 
		%-different from actual=[0.8404225]
		 accuracy=0.0
	input=[8.38068595 8.51779301 5.48893773] expected_output=[16.89847895824978] prediction=[2.6503491] 
		difference=[-14.248131] 
		%-different from actual=[0.8431605]
		 accuracy=0.0
	input=[9.19146332 6.84906628 5.48893773] expected_output=[16.04052959883995] prediction=[2.3977318] 
		difference=[-13.642797] 
		%-different from actual=[0.85052043]
		 accuracy=0.0
	input=[8.59507973 8.19229373 5.48893773] expected_output=[16.787373461220948] predic

		%-different from actual=[0.84202874]
		 accuracy=0.0
	input=[7.45645456 7.87359779 5.48893773] expected_output=[15.330052344861748] prediction=[2.3924413] 
		difference=[-12.937611] 
		%-different from actual=[0.8439378]
		 accuracy=0.0
	input=[7.55223729 6.78558765 5.48893773] expected_output=[14.337824932568731] prediction=[2.1716232] 
		difference=[-12.166202] 
		%-different from actual=[0.8485389]
		 accuracy=0.0
	input=[8.02420749 7.94307272 5.48893773] expected_output=[15.96728020305651] prediction=[2.480935] 
		difference=[-13.486345] 
		%-different from actual=[0.8446238]
		 accuracy=0.0
	input=[8.10016145 8.43533216 5.48893773] expected_output=[16.535493611872525] prediction=[2.5963068] 
		difference=[-13.939187] 
		%-different from actual=[0.8429858]
		 accuracy=0.0
	input=[8.73664995 9.019543   5.48893773] expected_output=[17.756192951573638] prediction=[2.804053] 
		difference=[-14.95214] 
		%-different from actual=[0.8420803]
		 accuracy=0.0
	input=[8.78889831 7.05444966

		%-different from actual=[0.836214]
		 accuracy=0.0
	input=[7.74109909 7.80710329 5.48893773] expected_output=[15.548202380161346] prediction=[2.4150863] 
		difference=[-13.133116] 
		%-different from actual=[0.844671]
		 accuracy=0.0
	input=[9.10609035 8.50593    5.48893773] expected_output=[17.61202034973991] prediction=[2.7418432] 
		difference=[-14.870177] 
		%-different from actual=[0.84431976]
		 accuracy=0.0
	input=[9.19166711 8.55775915 5.48893773] expected_output=[17.74942625987346] prediction=[2.7640476] 
		difference=[-14.985378] 
		%-different from actual=[0.844274]
		 accuracy=0.0
	input=[7.73368357 9.20943997 5.48893773] expected_output=[16.94312353750892] prediction=[2.7147422] 
		difference=[-14.22838] 
		%-different from actual=[0.8397732]
		 accuracy=0.0
	input=[7.61874238 8.66939912 5.48893773] expected_output=[16.288141501975982] prediction=[2.5840745] 
		difference=[-13.704067] 
		%-different from actual=[0.8413524]
		 accuracy=0.0
	input=[8.14438887 7.67508186 5.

		%-different from actual=[0.8419399]
		 accuracy=0.0
	input=[9.04746844 8.44333134 5.48893773] expected_output=[17.490799781821323] prediction=[2.7208247] 
		difference=[-14.769974] 
		%-different from actual=[0.8444425]
		 accuracy=0.0
	input=[8.53836743 8.74512526 5.48893773] expected_output=[17.283492686109888] prediction=[2.7195225] 
		difference=[-14.563971] 
		%-different from actual=[0.842652]
		 accuracy=0.0
	input=[9.10486874 9.13281117 5.48893773] expected_output=[18.237679908500894] prediction=[2.8760684] 
		difference=[-15.361612] 
		%-different from actual=[0.8423008]
		 accuracy=0.0
	input=[9.20452349 7.68017564 5.48893773] expected_output=[16.88469912709121] prediction=[2.5775876] 
		difference=[-14.307112] 
		%-different from actual=[0.84734184]
		 accuracy=0.0
	input=[6.76503898 9.01614864 5.48893773] expected_output=[15.781187619392282] prediction=[2.5477362] 
		difference=[-13.233452] 
		%-different from actual=[0.8385587]
		 accuracy=0.0
	input=[7.32251043 9.013960